In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scanpy as sc
import squidpy as sq
import gpflow
import tensorflow as tf
from GPcounts.GPcounts_Module import Fit_GPcounts
import statsmodels.formula.api as smf 
import statsmodels.api as sm

2023-03-09 18:25:33.942665: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/rs619065/miniconda3/envs/r-4.1/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [2]:
adata = sc.read_h5ad(f"../../data/simulation/svgs/adata_rep_0.h5ad")

In [3]:
Y = pd.DataFrame(data=adata.layers['counts'].todense(), 
                      index=adata.obs_names, 
                      columns=adata.var_names)

In [4]:
spatial_locations = pd.DataFrame(data=adata.obsm['spatial'], 
                                 index=adata.obs_names, columns=['x', 'y'])
spatial_locations['total_counts'] = adata.layers['counts'].sum(axis=1)

In [5]:
scales = []
for i in range(0, len(Y.columns)):
    model=smf.glm(formula = "Y.iloc[:, i]~0+spatial_locations['total_counts']", 
                  data=Y, 
                  family=sm.families.NegativeBinomial(sm.families.links.log())).fit()
    res = model.params[0]*spatial_locations['total_counts']
    scales.append(res)
    
scalesdf=pd.DataFrame(scales)
scalesdf=scalesdf.T

In [6]:
Y = Y.T

In [7]:
X = spatial_locations[['x', 'y']]
gp_counts = Fit_GPcounts(X, Y, scale=scalesdf, safe_mode=False)

In [8]:
log_likelihood_ratio = gp_counts.One_sample_test("Negative_binomial")

  0%|          | 0/250 [00:00<?, ?it/s]2023-03-09 18:25:39.385954: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


  0%|          | 0/250 [00:06<?, ?it/s]


TypeError: in user code:

    File "/home/rs619065/miniconda3/envs/r-4.1/lib/python3.8/site-packages/gpflow/optimizers/scipy.py", line 120, in _tf_eval  *
        loss, grads = _compute_loss_and_gradients(
    File "/home/rs619065/miniconda3/envs/r-4.1/lib/python3.8/site-packages/gpflow/optimizers/scipy.py", line 223, in _compute_loss_and_gradients  *
        loss = loss_closure()
    File "/home/rs619065/miniconda3/envs/r-4.1/lib/python3.8/site-packages/check_shapes/integration/tf.py", line 57, in wrapped_method  *
        return wrapped_function(self, *args, **kwargs)
    File "/home/rs619065/miniconda3/envs/r-4.1/lib/python3.8/site-packages/check_shapes/decorator.py", line 120, in wrapped_function  *
        return func(*args, **kwargs)
    File "/home/rs619065/miniconda3/envs/r-4.1/lib/python3.8/site-packages/gpflow/models/training_mixins.py", line 64, in training_loss  *
        return self._training_loss()  # type: ignore[attr-defined]
    File "/home/rs619065/miniconda3/envs/r-4

In [ ]:
results = gp_counts.calculate_FDR(log_likelihood_ratio)
results